<h2> IMPORT LIBRARY </h2>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout,SpatialDropout2D
from tensorflow.keras.models  import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img 
import numpy as np
import os
import time
import glob

D:\Anaconda\envs\tubes-tf\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tubes-tf\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tubes-tf\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tubes-tf\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: P

In [3]:
#Cek GPU yang akan dipakai
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3914156753891776696
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3220452147
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8063702982043046726
physical_device_desc: "device: 0, name: GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [2]:
os.listdir(os.path.join(os.getcwd(), "dataset-resized"))

['.DS_Store', 'cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

<h2>DATASET INPUT</h2>

In [5]:
path = "dataset-resized"
listImage = glob.glob(os.path.join(path, '*/*.jpg'))
print("Jumlah dataset: " + str(len(listImage)))

Jumlah dataset: 2527


<h2> IMAGE AUGMENTATION </h2>

Membuat image augmentation pada dataset yang sudah ada untuk memperbanyak jumlah dataset</br>
Membagi dataset yang ada menjadi dataset untuk training dan test </br>



In [14]:
train = ImageDataGenerator(horizontal_flip=True,
                          vertical_flip=True,
                          validation_split=0.1,
                          rescale=1./255,
                          shear_range=0.1,
                          zoom_range=0.1,
                          width_shift_range=0.1,
                          height_shift_range=0.1)

test = ImageDataGenerator(rescale=1/255, validation_split=0.1)

train_generator = train.flow_from_directory(path, 
                                            target_size=(300,300),
                                            batch_size=32,
                                            class_mode="categorical",
                                            subset="training")

test_generator = test.flow_from_directory(path,
                                          target_size=(300,300),
                                          batch_size=32,
                                          class_mode="categorical",
                                          subset="validation")

#label untuk image
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.
{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}


In [16]:
for img,label in train_generator:
    break
img.shape, label.shape

((32, 300, 300, 3), (32, 6))

In [17]:
Labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as file:
    file.write(Labels)

<h2> CNN (Convolutional Neural Network) </h2>

Membuat model CNN

In [18]:
model = Sequential()

model.add(Conv2D(32,(3,3), padding="same", input_shape=(300,300,3), activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(32, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6, activation="softmax"))

#menyimpan model
modelFile = "modelConfig.h5"
checkpoint1 = ModelCheckpoint(modelFile, monitor="val_acc", verbose=1, save_best_only=True, mode="max")
callback_list = [checkpoint1]

In [19]:
#konfigurasi model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 43808)             0         
__________

In [20]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["acc"])

<h2>TRAIN</h2>

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=120,
                              steps_per_epochs=2276//32,
                              validation_data=test_generator,
                              validation_steps=251//32,
                              workers=4,
                              callback=callbacks_list)